In [1]:
# Import modules
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
#load data and take out id columns
new_data = pd.read_csv('data/model_data.csv')
new_data1 = pd.read_csv('data/model_data.csv')
droplis = ['Unnamed: 0','encounter_id','patient_nbr']
new_data = new_data.drop(droplis, axis=1)

In [3]:
#find numeric columns
new_data._get_numeric_data().columns

Index(['admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses', 'target', 'self_pay',
       'medicaid/medicare', 'coverByInsurance', 'clustering3', 'clustering4',
       'clustering2'],
      dtype='object')

In [4]:
#numeric columns
num =['time_in_hospital','num_lab_procedures','num_procedures','num_medications','number_outpatient','number_emergency',
'number_inpatient','number_diagnoses']


In [5]:
#normalize new_data numeric columns
from sklearn.preprocessing import Normalizer
transformer = Normalizer().fit(new_data[num])
new_data[num] = transformer.transform(new_data[num])
new_data[num] = pd.DataFrame(new_data[num], columns= new_data1[num].columns)

In [6]:
#enlabel the columns
new_data = new_data
new_data['admission_type_id'] = pd.Categorical(new_data.admission_type_id)
new_data['discharge_disposition_id'] = pd.Categorical(new_data.discharge_disposition_id)
new_data['admission_source_id'] = pd.Categorical(new_data.admission_source_id)

cat_cols = list(new_data.select_dtypes(include=[object]).columns)
for col in cat_cols:
   new_data[col] = pd.Categorical(new_data[col])

le = preprocessing.LabelEncoder()

col_to_encode = new_data[list(new_data.select_dtypes(include=['category']).columns)]
for col in col_to_encode:
   new_data[col] = le.fit_transform(new_data[col])

In [7]:
#new_data['admission_type_id'] = pd.Categorical(new_data.admission_type_id)
#new_data['discharge_disposition_id'] = pd.Categorical(new_data.discharge_disposition_id)
#new_data['admission_source_id'] = pd.Categorical(new_data.admission_source_id)
#cat_cols = list(new_data.select_dtypes(include=[object]).columns)
#for col in cat_cols:
#   new_data[col] = pd.Categorical(new_data[col])

#enc = OneHotEncoder(handle_unknown='ignore')

#col_to_encode = new_data[list(new_data.select_dtypes(include=['category']).columns)]
#for col in col_to_encode:
#   new_data[col] = enc.fit_transform(new_data[col])

In [8]:
#Spliting to training and testing datasets
target = new_data.target
X_train, X_test, y_train, y_test = train_test_split(new_data, target, test_size=0.25)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(64875, 52) (64875,)
(21625, 52) (21625,)


In [9]:
##### Up-sample logistic regression  #####

In [10]:
#Up-sample X_train
from sklearn.utils import resample
train_x_majority = X_train[X_train.target==0]
train_x_minority = X_train[X_train.target==1]
train_x_minority_upsampled = resample(train_x_minority,replace=True,n_samples=57558,random_state=123)
x = pd.concat([train_x_majority, train_x_minority_upsampled])


In [11]:
#Up-sample y_train
y_train = pd.DataFrame(y_train)
train_y_majority = y_train[y_train.target==0]
train_y_minority = y_train[y_train.target==1]
train_y_minority_upsampled = resample(train_y_minority,replace=True,n_samples=57558,random_state=123)
y = pd.concat([train_y_majority, train_y_minority_upsampled])
y.target.value_counts()

0    57621
1    57558
Name: target, dtype: int64

In [12]:
#drop target from training
x = x.drop('target', axis=1)

In [13]:
## Fit the model with training
decision_logit = LogisticRegressionCV(cv=5, random_state=0,multi_class='multinomial')
decision_logit.fit(x, y)

## The score (accuracy for classification problems):
decision_logit.score(x, y)

/Users/jasonwang/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5845423210828363

In [14]:
#up-sample AUC for training 
from sklearn.metrics import roc_auc_score
upsample_AUC_train =roc_auc_score(y, decision_logit.predict_proba(x)[:,1])
upsample_AUC_train

0.6250483834928829

In [15]:
#drop target from test
X_test = X_test.drop('target', axis=1)

In [16]:
## Fit the model with testing
decision_logit1 = LogisticRegressionCV(cv=5, random_state=0, multi_class='multinomial')
decision_logit1.fit(X_test, y_test)

## The score (accuracy for classification problems):
decision_logit1.score(X_test, y_test)

0.8890635838150289

In [17]:
#AUC for testing
from sklearn.metrics import roc_auc_score
upsample_AUC_test=roc_auc_score(y_test, decision_logit.predict_proba(X_test)[:,1])
upsample_AUC_test

0.6190443008106945

In [18]:
########## up-sampled test

In [19]:
#Up-sample Minority Class
majority = new_data[new_data.target==0]
minority = new_data[new_data.target==1]
minority_upsampled = resample(minority,replace=True,n_samples=76847,random_state=123)
df_upsampled = pd.concat([majority, minority_upsampled])
df_upsampled.target.value_counts()

1    76847
0    76847
Name: target, dtype: int64

In [20]:
#Spliting to training and testing datasets
target = df_upsampled.target
X_train, X_test, y_train, y_test = train_test_split(df_upsampled, target, test_size=0.25)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(115270, 52) (115270,)
(38424, 52) (38424,)


In [21]:
#drop target from training
X_test = X_test.drop('target', axis=1)

In [22]:
## Fit the model with testing
decision_logit1 = LogisticRegressionCV(cv=5, random_state=0, multi_class='multinomial')
decision_logit1.fit(X_test, y_test)

## The score (accuracy for classification problems):
decision_logit1.score(X_test, y_test)

0.5910108265667291

In [23]:
#AUC for testing
from sklearn.metrics import roc_auc_score
upsample_AUC_test=roc_auc_score(y_test, decision_logit.predict_proba(X_test)[:,1])
upsample_AUC_test

0.6223170564707721